In [ ]:
#libs for reading data
import xarray as xr
import gcsfs
import glob
import numpy as np
import matplotlib.pyplot as plt
from xhistogram.xarray import histogram

#lib for dask gateway
from dask_gateway import Gateway
from dask.distributed import Client

In [ ]:
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=75)
client = Client(cluster)
cluster

In [ ]:
zstore = 'gs://pangeo-nasa-ccmp/zarr'

fs = gcsfs.GCSFileSystem(project='pangeo-181919',requester_pays=True)

ds = xr.open_zarr(fs.get_mapper(zstore))

ds['wspd']=np.sqrt(ds.uwnd**2+ds.vwnd**2)

ds

In [ ]:
ds.wspd[0,:,:].plot()

In [ ]:
ts = ds.sel(latitude=38.45,longitude=237.34,method='nearest').load()

In [ ]:
dy = ts.resample(time='1D').mean()
dy

In [ ]:
dy.time.dt.month

In [ ]:
mon = dy.time.dt.month
sub = dy.where((mon>=9) & (mon<=10))

In [ ]:
sub.uwnd.plot()

# where are the strong storms occuring?  is there any shift in latitudinal extent?

In [ ]:
d,x=[],[]
for iwnd in range(20,36,5)
    high_val = ds.wspd.where(ds.wspd>25)
    high_cnt = high_val/high_val
    high_lat = high_cnt.sum({'longitude'})
    high_lat_computed = high_lat.compute()
    d.append(high_lat_computed)
    x.append(iwnd)
mn_yr = xr.concat(m, dim='year')
mn_yr['year']=
glb_mn = np.mean(mn_yr)


In [ ]:
high_lat.transpose().plot(cmap='YlOrBr',vmin=0,vmax=10)
plt.savefig('../../figures/num_storms.png')

In [ ]:
# maybe era5 reanalysis doesn't see strong wind events, try something esle?
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml")
list(cat['atmosphere'])

In [ ]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/atmosphere.yaml")
ds  = cat["era5_hourly_reanalysis_single_levels_sa"].to_dask()
ds

In [ ]:
ds.u10[0,:,:].plot()

In [ ]:
#ts = ds.sel(lon=-122.66,lat=38.45,method='nearest')
ts = ds.sel(longitude=-237.34,latitude=38.45,method='nearest')

In [ ]:
dy = ts.resample(time='1D').mean().load()